# Test Dickens
This notebook replicates the functionality of the `test_ollama.py` script.

In [1]:
import os
from lightrag import LightRAG, QueryParam
from lightrag.llm.openai import gpt_4o_mini_complete, gpt_4o_complete, openai_embed
from lightrag.llm.ollama import ollama_model_complete, ollama_embed
from lightrag.llm.siliconcloud import siliconcloud_embedding
from lightrag.utils import EmbeddingFunc
import asyncio
from lightrag import LightRAG, QueryParam
from lightrag.llm.openai import openai_complete_if_cache
from lightrag.llm.siliconcloud import siliconcloud_embedding
from lightrag.utils import EmbeddingFunc
import numpy as np


# Uncomment the below two lines if running in a jupyter notebook to handle the async nature of rag.insert()
import nest_asyncio
nest_asyncio.apply()

Looking in indexes: https://mirrors.cloud.tencent.com/pypi/simple


ERROR: Could not find a version that satisfies the requirement lmdeploy (from versions: none)
ERROR: No matching distribution found for lmdeploy


In [2]:
WORKING_DIR = "./KG"

if not os.path.exists(WORKING_DIR):
    os.mkdir(WORKING_DIR)

# 在导入部分添加
from dotenv import load_dotenv

# 在代码的开头加载环境变量
load_dotenv()

True

In [3]:
async def llm_model_func(
    prompt, system_prompt=None, history_messages=[], keyword_extraction=False, **kwargs
) -> str:
    return await openai_complete_if_cache(
        "Qwen/Qwen2.5-7B-Instruct",
        prompt,
        system_prompt=system_prompt,
        history_messages=history_messages,
        api_key=os.getenv("SILICONFLOW_API_KEY"),
        base_url="https://api.siliconflow.cn/v1/",
        **kwargs,
    )


async def embedding_func(texts: list[str]) -> np.ndarray:
    return await siliconcloud_embedding(
        texts,
        model="netease-youdao/bce-embedding-base_v1",
        api_key=os.getenv("SILICONFLOW_API_KEY"),
        max_token_size=512,
    )



In [4]:
# function test
async def test_funcs():
    result = await llm_model_func("How are you?")
    print("llm_model_func: ", result)

    result = await embedding_func(["How are you?"])
    print("embedding_func: ", result)


asyncio.run(test_funcs())


llm_model_func:  I'm just a digital entity, so I don't have feelings, but I'm here and ready to help you with any questions or tasks you might have! How can I assist you today?
embedding_func:  [[ 3.82568017e-02  1.12520000e-02 -1.56639684e-02  1.11631686e-02
   9.46056377e-03 -5.98626165e-03  9.40134283e-03 -9.31251049e-03
  -1.66411158e-02 -1.13507025e-02 -5.73654622e-02 -9.91459191e-03
   8.42518210e-02 -7.01275980e-03 -2.97092292e-02  8.16757046e-03
   8.60185828e-03 -3.19793709e-02  1.01045927e-03  9.37173143e-03
   7.17364401e-02 -1.39860390e-02  2.33725756e-02  3.42495106e-02
   3.63617279e-02 -4.36508516e-03 -5.16703678e-03 -2.17760753e-04
  -2.05546408e-03 -9.23354924e-03 -2.51294672e-02  1.53086428e-02
   1.96317788e-02 -7.91588128e-02 -3.04593630e-02 -3.39040533e-03
  -4.94298413e-02 -1.93159338e-02  1.10703893e-01  8.81209299e-02
   2.80312989e-02  3.66380922e-02  2.40042675e-02 -3.75264063e-02
  -3.78619954e-02  9.24095220e-04  9.65796690e-03 -2.00364571e-02
   2.73009054e

In [6]:
rag = LightRAG(
    working_dir=WORKING_DIR,
    chunking_mode="markdown",
    llm_model_func=llm_model_func,
    embedding_func=EmbeddingFunc(
        embedding_dim=768, max_token_size=512, func=embedding_func
    ),
)


INFO:lightrag:Logger initialized for working directory: ./KG
INFO:lightrag:Load KV llm_response_cache with 6 data
INFO:lightrag:Load KV full_docs with 8 data
INFO:lightrag:Load KV text_chunks with 56 data
INFO:lightrag:Loaded graph from ./KG/graph_chunk_entity_relation.graphml with 856 nodes, 595 edges
INFO:nano-vectordb:Load (769, 768) data
INFO:nano-vectordb:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': './KG/vdb_entities.json'} 769 data
INFO:nano-vectordb:Load (595, 768) data
INFO:nano-vectordb:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': './KG/vdb_relationships.json'} 595 data
INFO:nano-vectordb:Load (56, 768) data
INFO:nano-vectordb:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': './KG/vdb_chunks.json'} 56 data
INFO:lightrag:Loaded document status storage with 8 records


In [ ]:
# Choose the model here
model_choice = "ollama"  # Options: "ollama", "openai"

llm_model_func = ollama_model_complete
embedding_func = EmbeddingFunc(
    embedding_dim=1024,
    max_token_size=8192,
    func=lambda texts: ollama_embed(
        texts,
        embed_model="bge-m3:latest"
    )
)
rag = LightRAG(
    working_dir=WORKING_DIR,
    llm_model_func=llm_model_func, 
    llm_model_name='qwen2.5:7b-instruct-q4_K_M',
    embedding_func=embedding_func,
    llm_model_max_token_size=32768,
    llm_model_kwargs={"host": "http://localhost:11434", "options": {"num_ctx": 32768}},

)

In [7]:
with open("./kg9.md") as f:
    rag.insert(f.read())

INFO:lightrag:Inserting 1 to doc_status
INFO:lightrag:Stored 1 new unique documents
INFO:lightrag:Number of batches to process: 1.
INFO:lightrag:Start processing batch 1 of 1.
INFO:lightrag:Inserting 1 to doc_status
INFO:lightrag:Inserting 4 to chunks
INFO:lightrag:Inserting 1 to full_docs
INFO:lightrag:Inserting 4 to text_chunks
INFO:lightrag:Non-embedding cached missed(mode:default type:extract)
INFO:lightrag:Non-embedding cached missed(mode:default type:extract)
INFO:lightrag:Non-embedding cached missed(mode:default type:extract)


Processing Chapter: # 中国铁路广州局集团有限公司关于公布《广州局集团公司旅客列车登乘卡管理办法》的通知
Processing Section: ## 广州局集团公司旅客列车登乘卡管理办法
Processing Subsection: ### 第一章 总 则
Processing Subsection: ### 第二章 登乘卡规定
Processing Subsection: ### 第三章 登乘卡管理


INFO:lightrag:Non-embedding cached missed(mode:default type:extract)
INFO:lightrag:Inserting 1 to llm_response_cache
INFO:lightrag:Non-embedding cached missed(mode:default type:extract)
INFO:lightrag:Inserting 1 to llm_response_cache
INFO:lightrag:Non-embedding cached missed(mode:default type:extract)
INFO:lightrag:Inserting 1 to llm_response_cache
INFO:lightrag:Non-embedding cached missed(mode:default type:extract)
INFO:lightrag:Inserting 1 to llm_response_cache
INFO:lightrag:Non-embedding cached missed(mode:default type:extract)
INFO:lightrag:Inserting 1 to llm_response_cache
INFO:lightrag:Inserting 1 to llm_response_cache
INFO:lightrag:Inserting 1 to llm_response_cache
INFO:lightrag:Inserting 1 to llm_response_cache
INFO:lightrag:Inserting 49 to entities
INFO:lightrag:Inserting 36 to relationships
INFO:lightrag:New entities or relationships extracted.
INFO:lightrag:Inserting 1 to doc_status
INFO:lightrag:Completed batch 1 of 1.
INFO:lightrag:Writing graph with 904 nodes, 631 edges
I

In [8]:
q = "登乘证的管理包括哪些内容"

In [9]:
# Perform naive search
print("naive.....................")
print(rag.query(q, param=QueryParam(mode="naive")))

INFO:lightrag:Non-embedding cached missed(mode:naive type:query)


naive.....................


INFO:lightrag:Inserting 1 to llm_response_cache


登乘证的管理主要包括以下几个方面：

一、客运部负责登乘卡管理系统的研发及维护，确保系统正常运作，并对各站段进行指导和监督。同时，客运部还负责与印刷厂对接印制登乘卡，并通过站段做好发放和回收工作。

二、劳卫部负责指导和监督相关单位确保在册劳务工数据信息的准确性，并审核劳务工身份及其登乘卡通勤区段的有关工作。

三、财务部（收入部）有权对登乘卡的请领、审核、发放、缴销以及使用情况进行监督检查，而且收入稽查人员可以查扣违规的登乘卡及相关证件、证明，并负责通报和处理违反规定的行为。

四、各业务部门负责收集各单位的办理需求，并审核这些信息的录入和日常管理。比如车辆部、电务部和运输部负责车辆及车载设备厂家人员的登乘卡申请，保卫部则负责保安公司的相关申请。

五、申请登乘证的单位需要负责信息录入和日常管理，确保信息的准确性。申请单位需依据相关规章制度审核劳务工的信息并进行严格初审，防止越界提交申请。

这些职责分工确保了登乘证管理的全面性和细致性，维护了站车的良好秩序。


In [ ]:
# Perform local search
print("local.....................")
print(rag.query(q, param=QueryParam(mode="local")))

In [ ]:
# Perform global search
print("global.....................")
print(rag.query(q, param=QueryParam(mode="global")))

In [ ]:
# Perform hybrid search
print("hybrid.....................")
print(rag.query(q, param=QueryParam(mode="hybrid")))

In [10]:
# Perform mix search (Knowledge Graph + Vector Retrieval)
print("mix.....................")
print(rag.query(q, param=QueryParam(mode="mix")))

INFO:lightrag:Non-embedding cached missed(mode:mix type:query)
INFO:lightrag:Non-embedding cached missed(mode:mix type:keywords)


mix.....................


INFO:lightrag:Inserting 1 to llm_response_cache
INFO:lightrag:Query nodes: 申请流程, 使用规定, 审核程序, 权限管理, top_k: 60, cosine: 0.2
INFO:lightrag:Query edges: 登乘证管理, 管理内容, top_k: 60, cosine: 0.2
INFO:lightrag:Global query uses 54 entites, 60 relations, 9 chunks
INFO:lightrag:Local query uses 60 entites, 62 relations, 6 chunks


广州局集团公司对登乘证的管理涉及多个方面，具体包括：

### 管理职责
#### 客运部
- 负责登乘卡管理系统的研发及维护，检查指导各站段系统更新维护情况。
- 负责登记高铁车站客服系统设备维护的路外公司工作人员、相关保洁单位管理人员、动车餐饮公司管理人员的登乘卡办理需求。
- 负责与印刷厂对接印制登乘卡，并组织登乘卡的发放和回收销毁工作，督促站段做好登乘卡日常验证管理工作。
- 负责登乘卡验证系统后台数据的更新及清理。

#### 劳卫部
- 指导、监督有关单位及时维护更新在册劳务工数据信息，确保准确性。
- 指导、监督有关单位做好劳务工身份及登乘卡通勤区段审核的相关工作。

#### 财务部（收入部）
有权对登乘卡的请领、审核、发放、缴销和使用情况进行监督检查。收入稽查人员有权查扣各种违规登乘卡及有关证件、证明，通报和处理违规使用登乘卡事项。

#### 各业务部门
负责收集并审核本系统各单位提报的办理需求，指导本系统各单位做好登乘卡办证信息录入及日常管理工作。具体部门如车辆部、电务部、运输部和保卫部分别负责不同类型人员的登乘卡办理需求的收集和审核。

#### 申请登乘卡单位
- 负责本单位登乘卡的信息录入及日常管理，建立严格的登乘卡请领、保管、回收、缴交等制度。
- 对本单位劳务工资质及相关信息进行确认，严格初审，提交的通勤劳务工信息应符合特定条件。
- 负责动态管理持卡者的信息，特别注意人员调入调出及离岗离职情况。

### 登乘卡类别
- 广深港高铁跨境动车组登乘卡（分为公用卡和个人卡）
- 旅客列车登乘卡（分为通勤登乘卡和维保登乘卡两种）

### 登乘卡限制
- 跨境登乘卡、通勤登乘卡、维保登乘卡仅限在集团公司管内车站进出。
- 严禁超区间使用，严禁乘坐外局或港铁担当的旅客列车。
- 通勤登乘卡、维保登乘卡均不办理签证，不安排座席仅限乘坐相应区段内集团公司担当的动车组列车二等座。

### 登乘卡使用
- 携带跨境登乘卡或通勤登乘卡时，需同时出示本人二代居民身份证。
- 使用维保登乘卡时，须同时出示本人二代居民身份证，在作业时间内可登乘列车。

### 参考文献
1. [KG] "广州局集团公司旅客列车登乘卡管理办法"
2. [DC] "本小节标题是：### 第三章 登乘卡管理"
3. [KG] "广州局集团公司涉及到登乘卡管理的具体部门职责分工